In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier

# --- 設定和載入資料 ---
DATA_PATH = 'train.csv'
TARGET_VARIABLE = 'Exited'
FEATURES = ['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance',
            'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
            'Geography']

try:
    df = pd.read_csv(DATA_PATH)
    df.dropna(inplace=True)
    print("資料載入成功。")
except FileNotFoundError:
    print(f"錯誤：找不到檔案 {DATA_PATH}。請確認路徑正確。")
    exit()

X = df[FEATURES].copy()
y = df[TARGET_VARIABLE]

# --- 特徵處理 ---
numerical_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']
categorical_features = ['Gender', 'Geography', 'NumOfProducts', 'HasCrCard', 'IsActiveMember']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ],
    remainder='drop'
)

# --- 模型訓練（修正類別不均） ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', HistGradientBoostingClassifier(
        learning_rate=0.05,
        max_depth=3,
        max_iter=200,
        random_state=42,
        class_weight='balanced'  # ⭐ 修正類別不均
    ))
])

model_pipeline.fit(X_train, y_train)
print("模型訓練完成。")
print("-" * 60)

# --- LTV 估算（保持你原本的公式） ---
df['Churn_Prob'] = model_pipeline.predict_proba(X)[:, 1]

NIM_RATE = 0.02
PRODUCT_PROFIT = 50.0
ACTIVE_CARD_PROFIT = 30.0
L_MAX = 10.0

df['ActiveCard_Flag'] = ((df['HasCrCard'] == 1) & (df['IsActiveMember'] == 1)).astype(int)

df['Annual_Profit'] = (
    (df['Balance'] * NIM_RATE) +
    (df['NumOfProducts'] * PRODUCT_PROFIT) +
    (df['ActiveCard_Flag'] * ACTIVE_CARD_PROFIT)
)

# Expected lifespan = 1/p，但防止除以 0
df['Expected_Lifespan'] = np.minimum(1 / np.maximum(df['Churn_Prob'], 1e-6), L_MAX)

df['LTV'] = df['Annual_Profit'] * df['Expected_Lifespan']

print(f"總 LTV 估算 (所有客戶): ${df['LTV'].sum():,.2f}")
print("-" * 60)

# --- ROI 最佳化模型（Profit Ranking） ---
def calculate_roi_optimal(data, retention_cost, retention_success_rate):
    """
    利潤最大化模型：不使用 threshold，完全使用 ENR 排序。
    ENR = LTV * P(churn) * SR - RC
    """
    df_eval = data.copy()

    df_eval['ENR'] = (
        df_eval['LTV'] * df_eval['Churn_Prob'] * retention_success_rate
    ) - retention_cost

    actionable = df_eval[df_eval['ENR'] > 0].copy()

    if actionable.empty:
        return None

    total_roi = actionable['ENR'].sum()

    return {
        'Actionable_Count': len(actionable),
        'Total_ROI': total_roi,
        'Total_Retention_Cost': len(actionable) * retention_cost,
        'Total_Expected_Return': total_roi + len(actionable) * retention_cost,
        'Data': actionable.sort_values(by='ENR', ascending=False)
    }

# --- 模擬參數 ---
USER_RETENTION_COST = 500.0
USER_SUCCESS_RATE = 0.20

# --- 執行 ROI 最佳化 ---
result = calculate_roi_optimal(
    df,
    retention_cost=USER_RETENTION_COST,
    retention_success_rate=USER_SUCCESS_RATE
)

print("🚀 基於 profit-ranking 的最佳 ROI 模型結果")
print("-" * 60)

if result is None:
    print("⚠️ 沒有任何 ENR > 0 的客戶，挽留全部為虧損方案。")
else:
    print(f"⭐ 可挽留客戶數：{result['Actionable_Count']}")
    print(f"⭐ 總淨 ROI：${result['Total_ROI']:,.2f}")
    print(f"挽留成本：${result['Total_Retention_Cost']:,.2f}")
    print(f"挽留後預期總收益：${result['Total_Expected_Return']:,.2f}")
    print("-" * 60)

    # 顯示前 20 位最有價值挽留的客戶
    print("📋 前 20 名 ENR 最高的客戶（最應挽留）:")
    print(result['Data'][['Churn_Prob', 'LTV', 'ENR']].head(20).to_markdown(index=False, floatfmt=".4f"))


資料載入成功。
模型訓練完成。
------------------------------------------------------------
總 LTV 估算 (所有客戶): $59,757,706.20
------------------------------------------------------------
🚀 基於 profit-ranking 的最佳 ROI 模型結果
------------------------------------------------------------
⭐ 可挽留客戶數：2532
⭐ 總淨 ROI：$170,441.92
挽留成本：$1,266,000.00
挽留後預期總收益：$1,436,441.92
------------------------------------------------------------
📋 前 20 名 ENR 最高的客戶（最應挽留）:
|   Churn_Prob |        LTV |      ENR |
|-------------:|-----------:|---------:|
|       0.4325 |  8920.0440 | 271.6462 |
|       0.1418 | 26799.7213 | 260.2210 |
|       0.3364 | 11292.6432 | 259.7287 |
|       0.9721 |  3892.0759 | 256.6862 |
|       0.4845 |  7778.7316 | 253.7290 |
|       0.2286 | 16469.7548 | 252.8571 |
|       0.8121 |  4635.5077 | 252.8571 |
|       0.7375 |  5104.3101 | 252.8571 |
|       0.6361 |  5867.8661 | 246.5086 |
|       0.7524 |  4950.5376 | 244.9799 |
|       0.4727 |  7874.9853 | 244.4546 |
|       0.6626 |  5609.1354 | 243.2878 

In [ ]:
import joblib

# -----------------------------------------------------------
# 修正說明：
# 1. 你在 Cell 1 定義的是 'model_pipeline'，不是 'final_tuned_model'
# 2. 你在 Cell 1 已經設定好 'preprocessor'，所以不需要 'FeatureEngineer'
# 3. 我們直接對完整資料 X 和 y 進行訓練
# -----------------------------------------------------------

print("正在使用完整資料集 (X, y) 重新訓練 Pipeline...")

# 1. 確保模型使用全部資料進行擬合
# Pipeline 會自動執行 Cell 1 定義的 OneHotEncoder 和數值處理
model_pipeline.fit(X, y)

# 2. 儲存模型
model_filename = 'customer_churn_bank_roi.pkl'
joblib.dump(model_pipeline, model_filename)

print(f"✅ 模型已成功儲存為：{model_filename}")
print("這個 .pkl 檔案現在包含了「資料預處理」與「模型預測」的所有步驟，可以直接用於部署。")

正在使用完整資料集 (X, y) 重新訓練 Pipeline...
✅ 模型已成功儲存為：ROI_churn_model_bank.pkl
這個 .pkl 檔案現在包含了「資料預處理」與「模型預測」的所有步驟，可以直接用於部署。
